## <b>TRATAMENTO DADOS DE VENDAS DE REMEDIOS</b>

> #### Como os dados são separados por mês e cada arquivo tem tamanho total de 800 MB, tive que rodar o código para cada mês e filtrar apenas os remédios necessários para o estudo.

In [1]:
# Importa bibliotecas
import pandas as pd
import numpy as np

In [12]:
# Importa os dados 

df2 = pd.read_csv('data/raw/EDA_Industrializados_202003.csv', encoding='iso8859-1', sep = ';', on_bad_lines='skip', low_memory=False)

In [13]:
# Exclui linhas com valores NaN 

df2.dropna(subset = ['UF_VENDA', 'PRINCIPIO_ATIVO','QTD_VENDIDA'], inplace=True)

In [14]:
# Exclui todas as colunas não utilizadas

df2.drop(columns = ['MUNICIPIO_VENDA', 'CONSELHO_PRESCRITOR', 'UNIDADE_MEDIDA', 'DESCRICAO_APRESENTACAO', 'UF_CONSELHO_PRESCRITOR', 'TIPO_RECEITUARIO', 'CID10', 'SEXO', 'IDADE', 'UNIDADE_IDADE'], inplace = True)
df2

,ANO_VENDA,MES_VENDA,UF_VENDA,PRINCIPIO_ATIVO,QTD_VENDIDA
0,2020,3,SP,RIFAMICINA SV SÓDICA,1
1,2020,3,SP,RIFAMICINA SV SÓDICA,1
2,2020,3,SP,RIFAMICINA SV SÓDICA,1
3,2020,3,SP,RIFAMICINA SV SÓDICA,1
4,2020,3,SP,RIFAMICINA SV SÓDICA,8
...,...,...,...,...,...
6087776,2020,3,BA,CLORIDRATO DE CIPROFLOXACINO,1
6087777,2020,3,BA,CLORIDRATO DE CIPROFLOXACINO,1
6087778,2020,3,BA,CLORIDRATO DE CIPROFLOXACINO,1
6087779,2020,3,BA,CLORIDRATO DE CIPROFLOXACINO + DEXAMETASONA,1


In [15]:
# Filtra os remédios do Kit Covid

df2_kitcovid = df2[df2.PRINCIPIO_ATIVO.str.contains("CLOROQUINA|IVERMECTINA|NITAZONAXIDA|AZITROMICINA")]
df2_kitcovid

,ANO_VENDA,MES_VENDA,UF_VENDA,PRINCIPIO_ATIVO,QTD_VENDIDA
96,2020,3,SP,SULFATO DE HIDROXICLOROQUINA,1
97,2020,3,SP,SULFATO DE HIDROXICLOROQUINA,7
1046,2020,3,SP,AZITROMICINA,1
1047,2020,3,SP,AZITROMICINA,2
1048,2020,3,SP,AZITROMICINA,1
...,...,...,...,...,...
6087201,2020,3,GO,AZITROMICINA DI-HIDRATADA,8
6087202,2020,3,GO,AZITROMICINA DI-HIDRATADA,1
6087203,2020,3,GO,AZITROMICINA DI-HIDRATADA,6
6087204,2020,3,GO,AZITROMICINA DI-HIDRATADA,3


In [16]:
pd.unique(df2_kitcovid['PRINCIPIO_ATIVO'])

array(['SULFATO DE HIDROXICLOROQUINA', 'AZITROMICINA',
       'AZITROMICINA DI-HIDRATADA', 'IVERMECTINA'], dtype=object)

In [17]:
# Cria uma nova coluna para juntar mês e ano

df2_kitcovid['DATA'] = df2_kitcovid['ANO_VENDA'].apply(str) + '-' + df2_kitcovid['MES_VENDA'].apply(str)
df2_kitcovid

C:\Users\Ana Beatriz\AppData\Local\Temp\ipykernel_12016\3883176386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_kitcovid['DATA'] = df2_kitcovid['ANO_VENDA'].apply(str) + '-' + df2_kitcovid['MES_VENDA'].apply(str)


,ANO_VENDA,MES_VENDA,UF_VENDA,PRINCIPIO_ATIVO,QTD_VENDIDA,DATA
96,2020,3,SP,SULFATO DE HIDROXICLOROQUINA,1,2020-3
97,2020,3,SP,SULFATO DE HIDROXICLOROQUINA,7,2020-3
1046,2020,3,SP,AZITROMICINA,1,2020-3
1047,2020,3,SP,AZITROMICINA,2,2020-3
1048,2020,3,SP,AZITROMICINA,1,2020-3
...,...,...,...,...,...,...
6087201,2020,3,GO,AZITROMICINA DI-HIDRATADA,8,2020-3
6087202,2020,3,GO,AZITROMICINA DI-HIDRATADA,1,2020-3
6087203,2020,3,GO,AZITROMICINA DI-HIDRATADA,6,2020-3
6087204,2020,3,GO,AZITROMICINA DI-HIDRATADA,3,2020-3


In [18]:
# Deleta as colunas de mês e ano e agrupa os dados de forma a somar a quantidade de caixas e frascos vendidos de cada remédio por estado

df2_kit_estados = df2_kitcovid.drop(columns = ['ANO_VENDA', 'MES_VENDA']).groupby(['DATA','UF_VENDA', 'PRINCIPIO_ATIVO']).sum().reset_index()
df2_kit_estados

,DATA,UF_VENDA,PRINCIPIO_ATIVO,QTD_VENDIDA
0,2020-3,AC,AZITROMICINA,357
1,2020-3,AC,AZITROMICINA DI-HIDRATADA,1979
2,2020-3,AC,SULFATO DE HIDROXICLOROQUINA,15
3,2020-3,AL,AZITROMICINA,1178
4,2020-3,AL,AZITROMICINA DI-HIDRATADA,5258
...,...,...,...,...
79,2020-3,SP,AZITROMICINA DI-HIDRATADA,245875
80,2020-3,SP,SULFATO DE HIDROXICLOROQUINA,4282
81,2020-3,TO,AZITROMICINA,1175
82,2020-3,TO,AZITROMICINA DI-HIDRATADA,5437


In [19]:
df2_kit_estados.dtypes

DATA               object
UF_VENDA           object
PRINCIPIO_ATIVO    object
QTD_VENDIDA         int64
dtype: object

In [20]:
# Transforma a data em datatime

df2_kit_estados['DATA'] = pd.to_datetime(df2_kit_estados['DATA'])
df2_kit_estados

,DATA,UF_VENDA,PRINCIPIO_ATIVO,QTD_VENDIDA
0,2020-03-01,AC,AZITROMICINA,357
1,2020-03-01,AC,AZITROMICINA DI-HIDRATADA,1979
2,2020-03-01,AC,SULFATO DE HIDROXICLOROQUINA,15
3,2020-03-01,AL,AZITROMICINA,1178
4,2020-03-01,AL,AZITROMICINA DI-HIDRATADA,5258
...,...,...,...,...
79,2020-03-01,SP,AZITROMICINA DI-HIDRATADA,245875
80,2020-03-01,SP,SULFATO DE HIDROXICLOROQUINA,4282
81,2020-03-01,TO,AZITROMICINA,1175
82,2020-03-01,TO,AZITROMICINA DI-HIDRATADA,5437


In [21]:
# Exporta o CSV para uso na próxima etapa

df2_kit_estados.to_csv("202003.csv", encoding = 'utf-8')